In [3]:
import marimo as mo

mo.md("# Welcome to Jargon Test! 🌊🍃")

_md()

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
jargon_dict = ["RNN", "neural network", "backpropagation", "gradient descent"]  # Your domain list
text = "The RNN, also called neural network, uses backpropagation for training, and use gradient desc."

# Embed
text_tokens = [w for w in text.split() if len(w) > 2]
text_embeds = model.encode(text_tokens)  # Chunk text
jargon_embeds = model.encode(jargon_dict)

# Fuzzy match
sims = cosine_similarity(text_embeds, jargon_embeds)
matches = np.where(sims > 0.70)  # Threshold for fuzzy jargon

jargon_matches = sorted({jargon_dict[i] for i in matches[1]})
print("Detected jargon:", jargon_matches)

/Users/leonardoheis/Source/repo/jargons_test/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Detected jargon: ['RNN', 'backpropagation', 'gradient descent', 'neural network']


In [6]:
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

jargon_dict = ["RNN", "neural network", "backpropagation", "gradient descent"]
text = "The RNN, also called neural network, uses backpropagation for training, and use gradient desc."

# Embed (batch all inputs together for efficiency)
text_tokens = [w for w in text.split() if len(w) > 2]
all_inputs = text_tokens + jargon_dict

response = client.embeddings.create(
    model="text-embedding-3-large",
    input=all_inputs
)

embeddings = [item.embedding for item in response.data]
text_embeds = np.array(embeddings[:len(text_tokens)])  # First N are text tokens
jargon_embeds = np.array(embeddings[len(text_tokens):])  # Rest are jargon

# Fuzzy match (identical to before)
sims = cosine_similarity(text_embeds, jargon_embeds)
matches = np.where(sims > 0.7)
jargon_matches = sorted({jargon_dict[i] for i in matches[1]})
print("Detected jargon:", jargon_matches)


Detected jargon: ['RNN', 'backpropagation']


/Users/leonardoheis/Source/repo/jargons_test/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/leonardoheis/Source/repo/jargons_test/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/leonardoheis/Source/repo/jargons_test/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [7]:
import json
import os
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY environment variable is required.")

client = OpenAI(api_key=api_key)

def replace_jargon_with_llm(text: str, jargon_terms: list[str]) -> tuple[str, dict[str, str]]:
    """Call OpenAI to swap detected jargon with predefined replacements."""
    replacements_catalog = {
        "RNN": "Recurrent Neural Net",
        "neural network": "artificial intelligence model",
        "backpropagation": "error correction process",
        "gradient descent": "iterative optimization step",
    }
    requested_replacements = {
        term: replacements_catalog[term]
        for term in jargon_terms
        if term in replacements_catalog
    }
    if not requested_replacements:
        return text, {}
    system_prompt = (
        "You are a meticulous copy editor. Replace only the exact jargon keys"
        " provided in the replacement_map with their mapped phrases."
        " Keep every other character, word, and formatting identical to the"
        " original text."
    )
    user_prompt = (
        "Original text:\n"
        f"{text}\n\n"
        "replacement_map (JSON):\n"
        f"{json.dumps(requested_replacements, ensure_ascii=False)}\n\n"
        "Return JSON with keys 'rewritten_text' (the text after applying the"
        " replacements) and 'applied_replacements' (the subset actually used)."
    )
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    raw_output = response.output_text.strip()
    try:
        payload = json.loads(raw_output)
    except json.JSONDecodeError as exc:
        raise ValueError(f"Model returned invalid JSON: {raw_output}") from exc
    rewritten = payload.get("rewritten_text", text)
    applied = payload.get("applied_replacements", requested_replacements)
    return rewritten, applied

rewritten_text, suggested_replacements = replace_jargon_with_llm(text, jargon_matches)

highlighted_text = rewritten_text
for plain in suggested_replacements.values():
    highlighted_text = highlighted_text.replace(plain, f"[REPLACED: {plain}]")

print("Original text:", text)
print("jargon Dictionary:", jargon_dict)
print("Suggested replacements:", suggested_replacements)
print("Rewritten text:", rewritten_text)
print("Rewritten text (highlighted):", highlighted_text)

Original text: The RNN, also called neural network, uses backpropagation for training, and use gradient desc.
jargon Dictionary: ['RNN', 'neural network', 'backpropagation', 'gradient descent']
Suggested replacements: {'RNN': 'Recurrent Neural Net', 'backpropagation': 'error correction process'}
Rewritten text: The Recurrent Neural Net, also called neural network, uses error correction process for training, and use gradient desc.
Rewritten text (highlighted): The [REPLACED: Recurrent Neural Net], also called neural network, uses [REPLACED: error correction process] for training, and use gradient desc.


In [8]:
summary_table = mo.ui.table({
    "Field": [
        "Original text",
        "Jargon dictionary",
        "Suggested replacements",
        "Rewritten text",
        "Rewritten text (highlighted)",
    ],
    "Value": [
        text,
        ", ".join(jargon_dict),
        json.dumps(suggested_replacements, ensure_ascii=False),
        rewritten_text,
        highlighted_text,
    ],
})
summary_table

table()